<h1>PPP2R1A: Qian et al. (2023)</h1>
<p>From <a href="https://pubmed.ncbi.nlm.nih.gov/37761890/">Novel Variants of PPP2R1A in Catalytic Subunit Binding Domain and Genotype–Phenotype Analysis in Neurodevelopmentally Delayed Patients</a>a></p>
https://doi.org/10.3390/genes14091750

 Furthermore, we found that pathogenic variants clustered in HEAT repeats V, VI and VII, and patients with the Met180Val/Thr variants had macrocephaly, severe ID and hypotonia, but no epilepsy, whereas those with Arg258 amino acid changes had microcephaly, while a few had epilepsy or feeding problems.

In [26]:
import phenopackets as PPKt
from google.protobuf.json_format import MessageToDict, MessageToJson, Parse, ParseDict
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from collections import defaultdict
from IPython.display import HTML, display
from pyphetools.creation import *
from pyphetools.validation import *
from pyphetools.visualization import *
import pyphetools
import re
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.8.28


In [2]:
parser = HpoParser()
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
PMID = "PMID:37761890"
title = "Novel Variants of PPP2R1A in Catalytic Subunit Binding Domain and Genotype-Phenotype Analysis in Neurodevelopmentally Delayed Patients"
metadata = MetaData(created_by="ORCID:0000-0002-0736-9199", pmid=PMID, pubmed_title=title)
metadata.default_versions_with_hpo(version=hpo_version)
print(f"HPO version {hpo_version}")

HPO version 2023-10-09


In [3]:
df = pd.read_excel("input/qian_2023.xlsx")

In [4]:
df.head()

,Individual,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,Age/Sex,11y/F,4y/M,18y/M,18y/F,12y/M,4y/M,3.5y/F,5m/F,27y/M,...,?/M,?/M,?/M,?/F,?/F,?/F,?/F,?/F,?/F,?/F
1,Variant,c.96C>G,c.421T>A,c.455C>T,c.532A>T,c.533C>A,c.536C>T,c.536C>T,c.536C>A,c.538A>G,...,c.533C>G,c.533C>A,c.536C>T,c.544C>T,c.544C>T,c.544C>T,c.656C>T,c.658G>A,c.658G>A,c.772C>A
2,Inheritance,De novo,De novo,De novo,De novo,De novo,De novo,De novo,De novo,De novo,...,De novo,De novo,De novo,De novo,De novo,De novo,De novo,De novo,De novo,De novo
3,Birth size,N,N,N,/,/,N,N,N,N,...,/,/,/,/,/,/,/,/,/,/
4,Height,N,N,N,N,N,-3SD,N,N,N,...,/,/,/,/,/,/,/,/,/,/


In [5]:
# convert to row-based format
dft = df.transpose()
dft.columns = dft.iloc[0]
dft.drop(dft.index[0], inplace=True)
dft['patient_id'] = dft.index
dft.head()

Individual,Age/Sex,Variant,Inheritance,Birth size,Height,Head circumference,DD/ID,Language delay,Motor delay,Epilepsy,Brain MR,Behavior problem,Hypotonia,Feeding problem,Hearing loss,Extremities/spine,Heart,PMID,patient_id
1,11y/F,c.96C>G,De novo,N,N,N,"ID, moderate",moderate,-,Moderate,N,"ASD, self-injury",+,-,-,Hyperm.,N,PMID:33106617,1
2,4y/M,c.421T>A,De novo,N,N,macro,"ID, severe",Moderate,+ (walk 4y9m),-,N,ADHD,+,GR,-,Hyperm.,N,PMID:33106617,2
3,18y/M,c.455C>T,De novo,N,N,N,IQ 86,Mild,-(walk 15m),-,N,ASD,-,-,-,Hyperm.,N,PMID:33106617,3
4,18y/F,c.532A>T,De novo,/,N,+5.1SD,"ID, moderate",mild,+(walk 2y),-,/,anxious,+,-,-,"cubitus valgus, kyphoscoliosis, et al",/,PMID:33106617,4
5,12y/M,c.533C>A,De novo,/,N,+2SD,"ID, severe",Severe,+(walk 5y),-,N,"ADHD, destructive",+,+,+,"Hyperm. scoliosis, et al",/,PMID:33106617,5


In [6]:
column_mapper_d = defaultdict(ColumnMapper)
dft.columns

Index(['Age/Sex', 'Variant', 'Inheritance', 'Birth size', 'Height',
       'Head circumference', 'DD/ID', 'Language delay', 'Motor delay',
       'Epilepsy', 'Brain MR', 'Behavior problem', 'Hypotonia',
       'Feeding problem', 'Hearing loss', 'Extremities/spine', 'Heart', 'PMID',
       'patient_id'],
      dtype='object', name=' Individual ')

In [7]:
#output = OptionColumnMapper.autoformat(df=dft, concept_recognizer=hpo_cr)
# print(output)

In [8]:
# Intrauterine growth retardation HP:0001511
birth_size_d = { 'SGA': 'Intrauterine growth retardation',
 'small': 'Intrauterine growth retardation'}
excluded_d = {'N': 'Intrauterine growth retardation'}
birth_size_Mapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=birth_size_d, excluded_d=excluded_d)
birth_size_Mapper.preview_column(dft['Birth size'])
column_mapper_d['Birth size'] = birth_size_Mapper

In [9]:
height_d = { '-3SD': 'Short stature',
 '-2.6SD': 'Short stature',
 '-2.5SD': 'Short stature',
 'SS': 'Short stature'}
excluded_d = {'N': 'Short stature', '+2SD': 'Short stature',}
heightMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=height_d, excluded_d=excluded_d)
heightMapper.preview_column(dft['Height'])
column_mapper_d['Height'] = heightMapper

In [10]:
head_circumference_d = {
 'macro': 'Macrocephaly',
 '+5.1SD': 'Macrocephaly',
 'micro': 'Microcephaly',
 '+3SD': 'Macrocephaly',
 '+5.2SD': 'Macrocephaly',
 '+3.4SD': 'Macrocephaly',
 '+2.5SD': 'Macrocephaly',
 '+3.9SD': 'Macrocephaly',
 '-2.4SD': 'Microcephaly',
 'Micro': 'Microcephaly',
 '-2.5SD': 'Microcephaly',
 '-3.1SD': 'Microcephaly',
 '-3.5SD': 'Microcephaly',
 '-3SD': 'Microcephaly'}

excluded_d = {'N': 'Abnormality of skull size',}
head_circumferenceMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=head_circumference_d, excluded_d=excluded_d)
head_circumferenceMapper.preview_column(dft['Head circumference'])
column_mapper_d['Head circumference'] = head_circumferenceMapper

In [11]:
# dd/id, code as Global developmental delay
dd_d = {'ID': 'Intellectual disability',
 'moderate': 'Intellectual disability, moderate',
 'severe': 'Intellectual disability, severe',
 'ID/DD': 'Global developmental delay',
 'DD': 'Global developmental delay',
 'mild': 'Intellectual disability, mild',
}
ddMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=dd_d)
ddMapper.preview_column(dft['DD/ID'])
column_mapper_d['DD/ID'] = ddMapper

In [12]:
language_delay_d = {'moderate': 'Delayed speech and language development',
 'Moderate': 'Delayed speech and language development',
 'Mild': 'Delayed speech and language development',
 'mild': 'Delayed speech and language development',
 'Severe': 'Delayed speech and language development',
 'No words': 'Absent speech',
 'severe': 'Delayed speech and language development',
 '+': 'Delayed speech and language development',
 '/ ( no word)': 'Absent speech',
 '+ (30 word)': 'Delayed speech and language development',
 '+ (No word)': 'Absent speech'}
language_delayMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=language_delay_d)
language_delayMapper.preview_column(dft['Language delay'])
column_mapper_d['Language delay'] = language_delayMapper

In [13]:
motor_delay_d = {
 '+ (walk 4y9m)': 'Motor delay',

 '+(walk 2y)': 'Motor delay',
 '+(walk 5y)': 'Motor delay',
 '+ (can’t walk)': 'Motor delay',
 '+ (walk 2y)': 'Motor delay',
 '+ (walk 3y)': 'Motor delay',
 '+ (walk 2.5y)': 'Motor delay',
 '+(can’t sit 4.5y)': 'Motor delay',
 '+ (walk 6y)': 'Motor delay',
 '/ (no walk)': 'Motor delay',
 '+': 'Motor delay',
 '+(walk 2.5y)': 'Motor delay',
 '+ (walk 23m)': 'Motor delay',
 '+(walk 3y)': 'Motor delay',
 '+ (can’t crawl/sit)': 'Motor delay'}
excluded_d = {'-': 'Motor delay', '-(walk 15m)': 'Motor delay',}
motor_delayMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=motor_delay_d, excluded_d=excluded_d)
motor_delayMapper.preview_column(dft['Motor delay'])
column_mapper_d['Motor delay'] = motor_delayMapper

In [14]:
epilepsy_d = {'Moderate': 'Seizure',
 'moderate': 'Seizure',
 '/': 'PLACEHOLDER',
 '+': 'Seizure',
 'Mild': 'Seizure',
 'severe': 'Seizure',
 '/(Once febrile seizure)': 'Febrile seizure (within the age range of 3 months to 6 years)'}
excluded_d = {'-': 'Seizure'}
epilepsyMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=epilepsy_d, excluded_d=excluded_d)
epilepsyMapper.preview_column(dft['Epilepsy'])
column_mapper_d['Epilepsy'] = epilepsyMapper

In [15]:
brain_mr_d = {
 'CCA': 'Agenesis of corpus callosum',
 'ventriculomegaly': 'Ventriculomegaly',
 'white matter diffuse thinning': 'Reduced cerebral white matter volume',
 'PVLM': 'Periventricular leukomalacia',
 'hippocampal dysplasia': 'Dysgenesis of the hippocampus',
 'CCH': 'Hypoplasia of the corpus callosum',
 'brain stem hypoplasia': 'Hypoplasia of the brainstem',
 'hydrocephalus': 'Hydrocephalus',
 'pachygyria et al': 'Pachygyria',
 'bilateral ventriculomegaly': 'Ventriculomegaly',
 'enlarged third ventricle': 'Dilated third ventricle',
 'hypoplastic brainstem': 'Hypoplasia of the brainstem',
 'Partial CCA': 'Partial agenesis of the corpus callosum',
 'lateral ventriculomegaly': 'Lateral ventricle dilatation'}
brain_mrMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=brain_mr_d)
brain_mrMapper.preview_column(dft['Brain MR'])
column_mapper_d['Brain MR'] = brain_mrMapper

In [16]:
behavior_problem_d = {'ASD': 'Autistic behavior',
 'self-injury': 'Self-injurious behavior',
 'ADHD': 'Attention deficit hyperactivity disorder',
 'anxious': 'Anxiety',
 #'destructive': 'PLACEHOLDER',
 #'Hand chews': 'PLACEHOLDER',
 'aggressive': 'Aggressive behavior',
 'stereotypic': 'Abnormal repetitive mannerisms',
 'Self-injury': 'Self-injurious behavior',
 'hyperactivity': 'Hyperactivity',
 'autistic': 'Autistic behavior',
 'stereotypy': 'Abnormal repetitive mannerisms',
 'Autistic behavior': 'Autistic behavior'}
excluded_d = { '-': 'Atypical behavior',}
behavior_problemMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=behavior_problem_d)
behavior_problemMapper.preview_column(dft['Behavior problem'])
column_mapper_d['Behavior problem'] = behavior_problemMapper

In [17]:
hypotonia_d = {'+': 'Hypotonia',
 'Moderate': 'Hypotonia',
 'severe': 'Hypotonia',
 'moderate': 'Hypotonia',
}
excluded_d = { '-': 'Hypotonia'}
hypotoniaMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=hypotonia_d)
hypotoniaMapper.preview_column(dft['Hypotonia'])
column_mapper_d['Hypotonia'] = hypotoniaMapper

In [18]:
feeding_problem_d = {'-': 'Feeding difficulties',
 'GR': 'Gastroesophageal reflux',
 '+': 'Feeding difficulties',
 'TPN': 'Dependency on parenteral nutrition',
 'G-tube': 'Tube feeding',
 'mild': 'Feeding difficulties',
 'PEG': 'Tube feeding'}
excluded_d = { '-': 'Feeding difficulties'}
feeding_problemMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=feeding_problem_d, excluded_d=excluded_d)
feeding_problemMapper.preview_column(dft['Feeding problem'])
column_mapper_d['Feeding problem'] = feeding_problemMapper

In [19]:
hearing_loss_d = {
 '+': 'Hearing impairment'}
excluded_d = {'-': 'Hearing impairment'}
hearing_lossMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=hearing_loss_d, excluded_d=excluded_d)
hearing_lossMapper.preview_column(dft['Hearing loss'])
column_mapper_d['Hearing loss'] = hearing_lossMapper

In [20]:
extremities_spine_d = {'Hyperm.': 'Joint hypermobility',
 'cubitus valgus': 'Cubitus valgus',
 'kyphoscoliosis': 'Kyphoscoliosis',
 'Hyperm. scoliosis': 'Scoliosis',
 'hip & knee dislocations': 'Knee dislocation',
 'thoracolumbar scoliosis': 'Thoracolumbar scoliosis',
 'scoliosis': 'Scoliosis',
 'Genu recurvatum': 'Genu recurvatum',
 'Scoliosis': 'Scoliosis',
 'hyperm.': 'Joint hypermobility'}
extremities_spineMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=extremities_spine_d)
extremities_spineMapper.preview_column(dft['Extremities/spine'])
column_mapper_d['Extremities/spine'] = extremities_spineMapper

In [21]:
heart_d = {
 'PDA': 'Patent ductus arteriosus',
 'ASD': 'Atrial septal defect',
 'PFO': 'Patent foramen ovale',
 'asymmetric LVH': 'Left ventricular hypertrophy',
 'PS': 'Pulmonic stenosis',
 'VSD': 'Ventricular septal defect',
 'BAV': 'Bicuspid aortic valve',
 'DAA': 'Ascending tubular aorta aneurysm'}
heartMapper = OptionColumnMapper(concept_recognizer=hpo_cr, option_d=heart_d)
heartMapper.preview_column(dft['Heart'])
column_mapper_d['Heart'] = heartMapper

In [22]:
variant_list = dft['Variant'].unique()
variant_d = {}
PPP2R1A_transcript = "NM_014225.6"
genome = "hg38"
vvalidator = VariantValidator(genome_build=genome, transcript=PPP2R1A_transcript)
for v in variant_list:
    var = vvalidator.encode_hgvs(v)
    variant_d[v] = var
print(f"We extracted {len(variant_d)} unique variants")

https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_014225.6%3Ac.96C>G/NM_014225.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_014225.6%3Ac.421T>A/NM_014225.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_014225.6%3Ac.455C>T/NM_014225.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_014225.6%3Ac.532A>T/NM_014225.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_014225.6%3Ac.533C>A/NM_014225.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_014225.6%3Ac.536C>T/NM_014225.6?content-type=application%2Fjson
https://rest.variantvalidator.org/VariantValidator/variantvalidator/hg38/NM_014225.6%3Ac.536C>A/NM_014225.6?content-type=application%2Fjson
https://rest.variantv

<h2>Create separate columns for age and sex</h2>

In [27]:
def get_age(entry):
    """
    The original data has entries such as 11y/F, 16m/M, and Neonate/M, but also  "/" and "?/F" 
    """
    hack_d = {"16m/M": "P1Y4M",
              "13m/M": "P1Y1M",
              "7m, F ": "P7M",
              "/": None
             }
    if entry in hack_d:
        # malformed entry in original data
        return hack_d.get(entry)
    elif entry.startswith("7m,"):
        return "P7M"
    fields = entry.split("/")
    if len(fields) != 2:
        print(f"[ERROR] Malformed Age/Sex entry: \"{entry}\"")
        return None
    age = fields[0]
    if age == "3.5y":
        return "P3Y6M"
    if re.match(r"\d+y", age):
        return f"P{age.upper()}"
    if re.match(r"\d+m", age):
        return f"P{age.upper()}"
    if age == "Neonate":
        return "P1D"
    return None

In [28]:
def get_sex(entry):
    if entry == "/":
        return None
    elif entry.startswith("7m,"):
        return "female"
    fields = entry.split("/")
    if len(fields) != 2:
        print(f"[ERROR] Malformed Age/Sex entry: \"{entry}\"")
        return None
    sex = fields[1].strip()
    if sex == "M":
        return "male"
    elif sex == "F":
        return "female"
    else:
        return None

In [29]:
dft["age"] = dft['Age/Sex'].apply(lambda x: get_age(x))
ageMapper = AgeColumnMapper.iso8601('age')
ageMapper.preview_column(dft['age']).head()

Could not parse None as ISO8601 period
Could not parse None as ISO8601 period
Could not parse None as ISO8601 period
Could not parse None as ISO8601 period
Could not parse None as ISO8601 period
Could not parse None as ISO8601 period
Could not parse None as ISO8601 period
Could not parse None as ISO8601 period
Could not parse None as ISO8601 period
Could not parse None as ISO8601 period
Could not parse None as ISO8601 period
Could not parse None as ISO8601 period
Could not parse None as ISO8601 period
Could not parse None as ISO8601 period
Could not parse None as ISO8601 period
Could not parse None as ISO8601 period


,original column contents,age
0,P11Y,P11Y
1,P4Y,P4Y
2,P18Y,P18Y
3,P12Y,P12Y
4,P3Y6M,P3Y6M


In [30]:
dft["sex"] = dft['Age/Sex'].apply(lambda x: get_sex(x))
sexMapper = SexColumnMapper(male_symbol='male', female_symbol='female', column_name='sex')
sexMapper.preview_column(dft['sex']).head()

Could not map sex symbol None
Could not map sex symbol None
Could not map sex symbol None
Could not map sex symbol None


,original column contents,sex
0,female,FEMALE
1,male,MALE
2,male,MALE
3,female,FEMALE
4,male,MALE


In [32]:
variantMapper = VariantColumnMapper(variant_d=variant_d, 
                                    variant_column_name='Variant',
                                    default_genotype="heterozygous")

In [34]:
encoder = CohortEncoder(df=dft, 
                        hpo_cr=hpo_cr, 
                        column_mapper_d=column_mapper_d, 
                        individual_column_name="patient_id", 
                        agemapper=ageMapper, 
                        sexmapper=sexMapper,
                        variant_mapper=variantMapper, 
                        pmid=PMID,
                        metadata=metadata)
hgs2 = Disease(disease_id='OMIM:616362', disease_label='Houge-Janssen syndrome 2')
encoder.set_disease(hgs2)

In [36]:
individuals = encoder.get_individuals()
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1)
qc = QcVisualizer(ontology=hpo_ontology, cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

Could not parse None as ISO8601 period
Could not map sex symbol None
Could not parse None as ISO8601 period
Could not map sex symbol None
Could not parse None as ISO8601 period
Could not map sex symbol None
Could not parse None as ISO8601 period
Could not map sex symbol None
Could not parse None as ISO8601 period
Could not parse None as ISO8601 period
Could not parse None as ISO8601 period
Could not parse None as ISO8601 period
Could not parse None as ISO8601 period
Could not parse None as ISO8601 period
Could not parse None as ISO8601 period
Could not parse None as ISO8601 period
Could not parse None as ISO8601 period
Could not parse None as ISO8601 period
Could not parse None as ISO8601 period
Could not parse None as ISO8601 period


Level,Error category,Count
ERROR,CONFLICT,2
WARNING,REDUNDANT,32


In [37]:
individuals = cvalidator.get_error_free_individual_list()
table = PhenopacketTable(individual_list=individuals, metadata=metadata)
display(HTML(table.to_html()))

Individual,Disease,Genotype,Phenotypic features
1 (FEMALE; P11Y),Houge-Janssen syndrome 2 (OMIM:616362),NM_014225.6:c.96C>G (heterozygous),"Intellectual disability, moderate (HP:0002342); Delayed speech and language development (HP:0000750); Seizure (HP:0001250); Autistic behavior (HP:0000729); Self-injurious behavior (HP:0100716); Hypotonia (HP:0001252); Feeding difficulties (HP:0011968); Joint hypermobility (HP:0001382); excluded: Intrauterine growth retardation (HP:0001511); excluded: Short stature (HP:0004322); excluded: Abnormality of skull size (HP:0000240); excluded: Motor delay (HP:0001270); excluded: Feeding difficulties (HP:0011968); excluded: Hearing impairment (HP:0000365)"
2 (MALE; P4Y),Houge-Janssen syndrome 2 (OMIM:616362),NM_014225.6:c.421T>A (heterozygous),"Macrocephaly (HP:0000256); Intellectual disability, severe (HP:0010864); Delayed speech and language development (HP:0000750); Motor delay (HP:0001270); Attention deficit hyperactivity disorder (HP:0007018); Hypotonia (HP:0001252); Gastroesophageal reflux (HP:0002020); Joint hypermobility (HP:0001382); excluded: Intrauterine growth retardation (HP:0001511); excluded: Short stature (HP:0004322); excluded: Seizure (HP:0001250); excluded: Hearing impairment (HP:0000365)"
3 (MALE; P18Y),Houge-Janssen syndrome 2 (OMIM:616362),NM_014225.6:c.455C>T (heterozygous),Delayed speech and language development (HP:0000750); Autistic behavior (HP:0000729); Feeding difficulties (HP:0011968); Joint hypermobility (HP:0001382); excluded: Intrauterine growth retardation (HP:0001511); excluded: Short stature (HP:0004322); excluded: Abnormality of skull size (HP:0000240); excluded: Motor delay (HP:0001270); excluded: Seizure (HP:0001250); excluded: Feeding difficulties (HP:0011968); excluded: Hearing impairment (HP:0000365)
4 (FEMALE; P18Y),Houge-Janssen syndrome 2 (OMIM:616362),NM_014225.6:c.532A>T (heterozygous),"Macrocephaly (HP:0000256); Intellectual disability, moderate (HP:0002342); Delayed speech and language development (HP:0000750); Motor delay (HP:0001270); Anxiety (HP:0000739); Hypotonia (HP:0001252); Feeding difficulties (HP:0011968); Cubitus valgus (HP:0002967); Kyphoscoliosis (HP:0002751); excluded: Short stature (HP:0004322); excluded: Seizure (HP:0001250); excluded: Feeding difficulties (HP:0011968); excluded: Hearing impairment (HP:0000365)"
5 (MALE; P12Y),Houge-Janssen syndrome 2 (OMIM:616362),NM_014225.6:c.533C>A (heterozygous),"Intellectual disability, severe (HP:0010864); Delayed speech and language development (HP:0000750); Motor delay (HP:0001270); Attention deficit hyperactivity disorder (HP:0007018); Hypotonia (HP:0001252); Feeding difficulties (HP:0011968); Hearing impairment (HP:0000365); Scoliosis (HP:0002650); excluded: Short stature (HP:0004322); excluded: Seizure (HP:0001250)"
6 (MALE; P4Y),Houge-Janssen syndrome 2 (OMIM:616362),NM_014225.6:c.536C>T (heterozygous),"Short stature (HP:0004322); Microcephaly (HP:0000252); Intellectual disability, severe (HP:0010864); Delayed speech and language development (HP:0000750); Motor delay (HP:0001270); Seizure (HP:0001250); Agenesis of corpus callosum (HP:0001274); Ventriculomegaly (HP:0002119); Hypotonia (HP:0001252); Dependency on parenteral nutrition (HP:0033994); Tube feeding (HP:0033454); Knee dislocation (HP:0004976); Kyphoscoliosis (HP:0002751); Patent ductus arteriosus (HP:0001643); excluded: Intrauterine growth retardation (HP:0001511)"
7 (FEMALE; P3Y6M),Houge-Janssen syndrome 2 (OMIM:616362),NM_014225.6:c.536C>T (heterozygous),"Microcephaly (HP:0000252); Intellectual disability, severe (HP:0010864); Absent speech (HP:0001344); Motor delay (HP:0001270); Agenesis of corpus callosum (HP:0001274); Hypotonia (HP:0001252); excluded: Intrauterine growth retardation (HP:0001511); excluded: Short stature (HP:0004322); excluded: Seizure (HP:0001250)"
8 (FEMALE; P5M),Houge-Janssen syndrome 2 (OMIM:616362),NM_014225.6:c.536C>A (heterozygous),Microcephaly (HP:0000252); Intellectual disability (HP:0001249); Globa

In [38]:
output_directory = "phenopackets"
Individual.output_individuals_as_phenopackets(individual_list=individuals, 
                                             metadata=metadata,
                                             outdir=output_directory)

We output 60 GA4GH phenopackets to the directory phenopackets


In [ ]:
# pxf validate --hpo hp.json *.json
# No errors